In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display

In [2]:
import IPython.display

In [3]:
#위까지가 멜스펙토그램 좀 다뤄본것. 큰의미는 없을듯
import pandas as pd
genre_gn_all=pd.read_json("/content/drive/My Drive/Kakao Arena/genre_gn_all.json",typ='series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
print(genre_gn_all)#전체장르
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] == '00']
print(gnr_code)#대분류장르
dtl_gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)
print(dtl_gnr_code)#세부장르

    gnr_code gnr_name
0     GN0100      발라드
1     GN0101   세부장르전체
2     GN0102      '80
3     GN0103      '90
4     GN0104      '00
..       ...      ...
249   GN2900      뮤지컬
250   GN2901   세부장르전체
251   GN2902    국내뮤지컬
252   GN2903    국외뮤지컬
253   GN3000    크리스마스

[254 rows x 2 columns]
    gnr_code    gnr_name
0     GN0100         발라드
6     GN0200          댄스
12    GN0300        랩/힙합
18    GN0400    R&B/Soul
22    GN0500        인디음악
32    GN0600        록/메탈
39    GN0700        성인가요
50    GN0800      포크/블루스
56    GN0900         POP
65    GN1000        록/메탈
80    GN1100      일렉트로니카
90    GN1200        랩/힙합
100   GN1300    R&B/Soul
109   GN1400  포크/블루스/컨트리
117   GN1500         OST
128   GN1600         클래식
144   GN1700          재즈
155   GN1800        뉴에이지
163   GN1900       J-POP
176   GN2000        월드뮤직
187   GN2100         CCM
198   GN2200      어린이/태교
215   GN2300        종교음악
218   GN2400          국악
225   GN2500         아이돌
232   GN2600      일렉트로니카
237   GN2700         EDM
242   GN2800

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
gnr_code = gnr_code.assign(join_code = gnr_code['gnr_code'].str[0:4])
dtl_gnr_code = dtl_gnr_code.assign(join_code = dtl_gnr_code['dtl_gnr_code'].str[0:4])

# Merge
gnr_code_tree = pd.merge(gnr_code, dtl_gnr_code, how = 'left', on = 'join_code')

gnr_code_tree[['gnr_code', 'gnr_name', 'dtl_gnr_code', 'dtl_gnr_name']]
gnr_code_tree[gnr_code_tree['gnr_code'] == 'GN0900']
#장르 트리.

,gnr_code,gnr_name,join_code,dtl_gnr_code,dtl_gnr_name
48,GN0900,POP,GN09,GN0901,세부장르전체
49,GN0900,POP,GN09,GN0902,얼터너티브팝
50,GN0900,POP,GN09,GN0903,올디스
51,GN0900,POP,GN09,GN0904,월드팝
52,GN0900,POP,GN09,GN0905,'60-'70
53,GN0900,POP,GN09,GN0906,'80-'90
54,GN0900,POP,GN09,GN0907,'00
55,GN0900,POP,GN09,GN0908,'10-


In [5]:
import os
song_meta = pd.read_json('/content/drive/My Drive/Kakao Arena/song_meta.json', typ = 'frame')
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


In [6]:
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  707989 non-null  object
 1   issue_date              707989 non-null  int64 
 2   album_name              707985 non-null  object
 3   album_id                707989 non-null  int64 
 4   artist_id_basket        707989 non-null  object
 5   song_name               707989 non-null  object
 6   song_gn_gnr_basket      707989 non-null  object
 7   artist_name_basket      707989 non-null  object
 8   id                      707989 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 48.6+ MB


In [7]:
a=song_meta.loc[song_meta['id']==325514]
print(a) #곡의 아이디 기반한 정보
a.info()
print(a.iloc[0]['album_id']) #그 후 이런식으로 꺼내면 됨. id가 325514인 곡의 앨범 아이디.
print(song_meta.loc[song_meta['album_id']==40432])#위 아이디로 곡들 불러옴
print(list(song_meta.loc[song_meta['album_id']==40432].iloc[:]['id']))#이렇게 325514 곡과 같은 앨범에 수록된 곡들을 불러올 수 있다.

artists=a.iloc[0]['artist_id_basket']
for artist in artists:
  b=song_meta.loc[song_meta['artist_id_basket'].apply(lambda x: artist in x)]
  print(b) #artist가 해당되는 곡들 불러옴
  print(list(b.iloc[:]['id']))#이렇게 325514 곡과 같은 아티스트가 만든 곡들을 불러올 수 있다.


print(song_meta.loc[song_meta['album_name'].str.match("그남자 그여자").fillna(True)])




       song_gn_dtl_gnr_basket  issue_date  ... artist_name_basket      id
325514       [GN0204, GN0201]    20031001  ...        [다나 (DANA)]  325514

[1 rows x 9 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 325514 to 325514
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   song_gn_dtl_gnr_basket  1 non-null      object
 1   issue_date              1 non-null      int64 
 2   album_name              1 non-null      object
 3   album_id                1 non-null      int64 
 4   artist_id_basket        1 non-null      object
 5   song_name               1 non-null      object
 6   song_gn_gnr_basket      1 non-null      object
 7   artist_name_basket      1 non-null      object
 8   id                      1 non-null      int64 
dtypes: int64(3), object(6)
memory usage: 80.0+ bytes
40432
       song_gn_dtl_gnr_basket  issue_date  ... artist_name_basket      id
86671        [

In [8]:
song_meta['artist_id_basket'].str.contains('2')

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
707984   NaN
707985   NaN
707986   NaN
707987   NaN
707988   NaN
Name: artist_id_basket, Length: 707989, dtype: float64

In [9]:
train = pd.read_json('/content/drive/My Drive/Kakao Arena/train.json', typ = 'frame')
print(train)

                                                     tags  ...                updt_date
0                                                     [락]  ...  2013-12-19 18:36:19.000
1                                                [추억, 회상]  ...  2014-12-02 16:19:42.000
2                                               [까페, 잔잔한]  ...  2017-08-28 07:09:34.000
3       [연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...  ...  2019-12-05 15:15:18.000
4                                                    [댄스]  ...  2011-10-25 13:54:56.000
...                                                   ...  ...                      ...
115066               [록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]  ...  2020-04-17 04:31:11.000
115067                                               [일렉]  ...  2015-12-24 17:23:19.000
115068             [담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]  ...  2019-08-16 20:59:22.000
115069                      [잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]  ...  2019-10-25 23:40:42.000
115070                          

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115071 non-null  object
 1   id            115071 non-null  int64 
 2   plylst_title  115071 non-null  object
 3   songs         115071 non-null  object
 4   like_cnt      115071 non-null  int64 
 5   updt_date     115071 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.3+ MB


In [11]:
plylst_song_map = train[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

In [12]:
# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag_map = train[['id', 'tags']]

# unnest tags
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

In [13]:
print(plylst_song_map)
print(plylst_tag_map)

             id   songs
0         61281  525514
1         61281  129701
2         61281  383374
3         61281  562083
4         61281  297861
...         ...     ...
5285866  100389  111365
5285867  100389   51373
5285868  100389  640239
5285869  100389   13759
5285870  100389  154078

[5285871 rows x 2 columns]
            id  tags
0        61281     락
1        10532    추억
2        10532    회상
3        76951    까페
4        76951   잔잔한
...        ...   ...
476326  131982   퇴근길
476327  100389  노래추천
476328  100389  팝송추천
476329  100389    팝송
476330  100389  팝송모음

[476331 rows x 2 columns]


In [14]:
test = pd.read_json('/content/drive/My Drive/Kakao Arena/test.json', typ = 'frame')#구현의 편의를 위해 test란 이름 하에 val파일로 바꿈
print(test)
test.info()

                           tags      id  ... like_cnt                updt_date
0                            []   70107  ...        6  2012-09-29 01:57:26.000
1      [나만의Best3, 인디아티스트들의추천음악]    7461  ...        0  2019-12-17 14:06:45.000
2                        [드라이브]   90348  ...       21  2015-05-23 10:44:48.000
3                         [분위기]   58617  ...        0  2019-03-14 09:47:34.000
4                            []  102395  ...       38  2018-07-11 16:43:32.000
...                         ...     ...  ...      ...                      ...
10735                      [추억]  137930  ...       16  2016-04-18 11:02:09.000
10736  [띵곡의, 우울, 분위기, 드라이브, 산책]     936  ...        1  2020-04-08 07:15:59.000
10737                    [기분전환]  110589  ...        6  2016-06-29 00:57:21.000
10738                      [여름]    2605  ...        4  2015-06-06 09:52:01.000
10739                        []  126651  ...       14  2015-10-26 17:43:30.000

[10740 rows x 6 columns]
<class 'pandas.core.frame.

In [15]:
print(test.shape[0])

print(test.loc[55,'songs'])
print(test.loc[55,'tags'])
print(test.loc[55,'plylst_title'])
print(test.loc[55,'like_cnt'])
print(test.loc[55,'updt_date'])

10740
[500538, 390336, 28563, 605152, 698661, 649722, 376097, 141595, 425511, 26633, 701099, 217200, 609220, 252254, 127432, 467385, 119826]
[]

13
2010-02-17 23:20:27.000


In [16]:
import re
title=test.loc[1,'plylst_title']
title=re.sub('[^A-Za-z0-9\u3130-\u318F\uAC00-\uD7A3]', " ", title)
#print(title)   팝 일렉 신나는 분위기를 원할 때
titleword=title.split(' ')
#print(titleword)  ['', '팝', '일렉', '신나는', '분위기를', '원할', '때']
resword=list(filter(('').__ne__, titleword))
print(resword)  #['팝', '일렉', '신나는', '분위기를', '원할', '때']

[]


In [17]:
print(plylst_song_map.loc[plylst_song_map['songs'] == '196298'])
print(plylst_song_map.loc[plylst_song_map['songs'] == '196298']['id'].tolist())
print(train.loc[train['id']==29392])
print(train['tags'])
lists=train.to_numpy()
print(lists.shape)
print(lists[0])

a=train.loc[train['id']==29392]
print(a.columns)
print(a.iloc[0]['songs'])
print(a.loc[:,'tags'])

             id   songs
666446    29392  196298
999836   122326  196298
2295467  105475  196298
2560885   81398  196298
2950411   83791  196298
3191987   66963  196298
3317281   34970  196298
4133936   79393  196298
4805430   38126  196298
['29392', '122326', '105475', '81398', '83791', '66963', '34970', '79393', '38126']
                tags     id  ... like_cnt                updt_date
14522  [힐링, 휴식, 잔잔한]  29392  ...        1  2015-01-27 02:52:39.000

[1 rows x 6 columns]
0                                                       [락]
1                                                  [추억, 회상]
2                                                 [까페, 잔잔한]
3         [연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...
4                                                      [댄스]
                                ...                        
115066                 [록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]
115067                                                 [일렉]
115068               [담시, 가족, 눈물, 그리움, 주

In [19]:
#list를 저장하는 코드. 런타임 관련 issue로 이와 같은 구현을 합니다.
#아래에 있는 list를 불러오는 코드와 저장경로를 잘 일치시켜주어야 합니다. 
import pickle
from tqdm import tqdm
tags_answer=[]
songs_answer=[]
lists=train.to_numpy() # (115071,6) size의 구조
a=0
for i in tqdm(range(8999,test.shape[0]),position=0, leave=True):#playlist 하나씩 도는거임 #현재 몇개의 valid파일이 test로 쓰여있다
  if (i%100==99 and a!=0):
    with open('/content/drive/My Drive/Kakao Arena/test/test'+str(i-99)+'~'+str(i)+'taglist.txt', 'wb') as f:
      pickle.dump(tags_answer, f)
    with open('/content/drive/My Drive/Kakao Arena/test/test'+str(i-99)+'~'+str(i)+'songlist.txt', 'wb') as f2:
      pickle.dump(songs_answer, f2)  
    tags_answer=[]
    songs_answer=[]#메모리 관련 이슈로 초기화.
    #100~199와 같은 식으로 파일 이름이 되어있는데 실제로는 99~198일 것 같음.. 큰 상관은 없으나 고려해야할듯

  tags_dict={}
  songs_dict={}
  a=a+1
  plylst_title=test.loc[i,'plylst_title']
  title=re.sub('[^A-Za-z0-9\u3130-\u318F\uAC00-\uD7A3]', " ", plylst_title)
  titleword=title.split(' ')
  resword=list(filter(('').__ne__, titleword))
  for tags in resword:#tags로 써있긴 하지만 플레이리스트 제목에 기반해서 사용.
    for lst in lists:
      if (tags in lst[0]):
        for traintags in lst[0]:
          traintags=str(traintags)+'titleplylst'
          if traintags in tags_dict:
            tags_dict[traintags]+=1
          else:
            tags_dict[traintags]=1
        for trainsongs in lst[3]:
          trainsongs=str(trainsongs)+'titleplylst'
          if trainsongs in songs_dict:
            songs_dict[trainsongs]+=1
          else:
            songs_dict[trainsongs]=1

  for songs in test.loc[i,'songs']:#플레이 리스트마다 50번 도는거임. 50번 아닐수도 있더군요..
    now_metadata=song_meta.loc[song_meta['id']==songs]
    now_albumid=now_metadata.iloc[0]['album_id']
    albumsong_list=list(song_meta.loc[song_meta['album_id']==now_albumid].iloc[:]['id'])
    for trainsongs in albumsong_list:
      trainsongs=str(trainsongs)+'albumsong'
      if trainsongs in songs_dict:
        songs_dict[trainsongs]+=1
      else:
        songs_dict[trainsongs]=1

    now_artistids=now_metadata.iloc[0]['artist_id_basket']

    for artist in now_artistids:
      now_artistsongs=song_meta.loc[song_meta['artist_id_basket'].apply(lambda x: artist in x)]
      artistsong_list=list(now_artistsongs.iloc[:]['id'])
      for trainsongs in artistsong_list:
        trainsongs=str(trainsongs)+'artistsong'
        if trainsongs in songs_dict:
          songs_dict[trainsongs]+=1
        else:
          songs_dict[trainsongs]=1
    plylsts_songs=plylst_song_map.loc[plylst_song_map['songs'] == str(songs)]#이 곡을 가지고 있는 플레이 리스트의 song_map 데이터프레임.
    if not plylsts_songs.empty:#비어있으면 별도의 작업을 하지 않음.. 머리가 아픕니다 근데 exception안해도 그냥 돌아가는 듯 보입니다.
      id_lists=plylsts_songs['id'].tolist()
      for ids in id_lists:# 이 아이디들은 playlist의 아이디들이다. str으로 주어진다.
        id_train=train.loc[train['id']==int(ids)]
        for trainsongs in id_train.iloc[0]['songs']:
          trainsongs=str(trainsongs)+'songplylst'
          if trainsongs in songs_dict:
            songs_dict[trainsongs]+=1
          else:
            songs_dict[trainsongs]=1
        for traintags in id_train.iloc[0]['tags']:
          traintags=str(traintags)+'songplylst'
          if traintags in tags_dict:
            tags_dict[traintags]+=1
          else:
            tags_dict[traintags]=1
    
    """
    a=song_meta.loc[song_meta['id']==325514]
print(a) #곡의 아이디 기반한 정보
a.info()
print(a.iloc[0]['album_id']) #그 후 이런식으로 꺼내면 됨. id가 325514인 곡의 앨범 아이디.
print(song_meta.loc[song_meta['album_id']==40432])#위 아이디로 곡들 불러옴
print(list(song_meta.loc[song_meta['album_id']==40432].iloc[:]['id']))#이렇게 325514 곡과 같은 앨범에 수록된 곡들을 불러올 수 있다.

artists=a.iloc[0]['artist_id_basket']
for artist in artists:
  b=song_meta.loc[song_meta['artist_id_basket'].apply(lambda x: artist in x)]
  print(b) #artist가 해당되는 곡들 불러옴
  print(list(b.iloc[:]['id']))#이렇게 325514 곡과 같은 아티스트가 만든 곡들을 불러올 수 있다.


print(song_meta.loc[song_meta['album_name'].str.match("그남자 그여자").fillna(True)])
    """
    
  # 위 포문은 리스트에 있는 곡들을 포함하는 다른 플레이리스트를 train에서 찾고,
  # 그 플레이 리스트의 태그와 곡들을 넣는겁니다.
  #songplylst가 붙은 키가 이 방식으로 가감이 된 것을 의미합니다.

  #print(tags_dict)
  #print(songs_dict)
  #print("다음 값은 tags도 고려됩니다 a=1")

  a=1#원래 파라미터 관해서 커버하려고 했는데 방식이 바뀌어서 그냥 1임
  for tags in test.loc[i,'tags']:#플레이리스트 마다 플레이리스트의 태그를 가져옴.
    #주의할점은 여기서는 np array를 통해 직접 들고옵니다 string list가지고 핸들링하는거 어려움..
    #프린트를 해보니 태그쪽이 수가 곡보다 적음에도 일반적으로 영향력이 3~4배정도 커보임 수록곡 1등인 밤편지가 2000개 리스트에 있는데
    #사랑 이런 태그는 막 10000개 가까이 있지 않을까 예상중. 일단은 parameter수정없이 함.
    for lst in lists:
      if (tags in lst[0]):
        for traintags in lst[0]:
          traintags=str(traintags)+'tagplylst'
          if traintags in tags_dict:
            tags_dict[traintags]+=a
          else:
            tags_dict[traintags]=a
        for trainsongs in lst[3]:
          trainsongs=str(trainsongs)+'tagplylst'
          if trainsongs in songs_dict:
            songs_dict[trainsongs]+=a
          else:
            songs_dict[trainsongs]=a
    
  tags_answer.append(tags_dict)
  songs_answer.append(songs_dict)
  #print(tags_dict)
  #print(songs_dict)
with open('/content/drive/My Drive/Kakao Arena/test/test'+str(i-99)+'~'+str(i)+'taglist.txt', 'wb') as f:
  pickle.dump(tags_answer, f)
with open('/content/drive/My Drive/Kakao Arena/test/test'+str(i-99)+'~'+str(i)+'songlist.txt', 'wb') as f2:
  pickle.dump(songs_answer, f2)  
  """
    print(train.loc[train['id']==10532])
    a=train.loc[train['id']==10532]
    print(a.loc[1,'songs'])
    print(a.loc[1,'tags'])
  """

100%|██████████| 1741/1741 [6:26:09<00:00, 13.31s/it]


[]
[]
[529437, 516103, 360067, 705713, 226062, 370891, 344934, 495061, 297884, 180274, 614059, 175436, 110325, 314668]
[589668, 21711, 570151, 320043, 13930, 599327, 216696, 423594, 170279, 93151, 455185, 634366, 277910, 671805, 622636, 232676, 442352]
[672718, 121924, 102694, 683657, 201558, 385110, 610734, 83940]
[173033, 138237, 63140, 15733, 461780, 308519, 645777, 225090, 73748, 204150]


In [20]:
#저장된 리스트를 불러와서 처리 후 json파일로 처리하는 코드입니다.
#Colab에서 나누어진 block순서로 차례대로 사용하는 것이 좋습니다만, 타 환경에서 한번에 구동해도 구동은 됩니다.
from tqdm import tqdm
import pickle
import json

tagplylst_parameter=10
songplylst_parameters=[30]
titleplylst_parameters=[50]
albumsong_parameters=[3]
artistsong_parameters=[3]
#list로 되어있는 parameter는 grid search구현이 가능하도록 되어있습니다. 따라 파일이름이 10305033results.json.. 과 같은 형태로 나옵니다.
before_songs=[362966, 576559, 514079, 427902, 272180, 110239, 135447, 472490, 106573, 369127, 163377, 18418, 228263, 347448, 42317, 252577, 317711, 360373, 699518, 64641, 200687, 54755, 496394, 108207, 124823, 524956, 658227, 305183, 11093, 335756, 74638, 704707, 518420, 528397, 191236, 349270, 161288, 529748, 618742, 448682, 108003, 569401, 443914, 646941, 254865, 204439, 200059, 342798, 205179, 675874, 275121, 277034, 498452, 659226, 521330, 580326, 119974, 3805, 699874, 704752, 114119, 532044, 584067, 133190, 563098, 93143, 318530, 493762, 603633, 420396, 347585, 191915, 222272, 130199, 355939, 259795, 581799, 553166, 493804, 423821, 410890, 307179, 195174, 475599, 19364, 375431, 405604, 445170, 436741, 378434, 588471, 459256, 138486, 32570, 52904, 417071, 654112, 643498, 455744, 146989]
before_tags=["kpop", "BTS", "스트레스", "방탄", "신나는", "기분전환", "드라이브", "여행", "운동", "감성"]
tags='추억' # 가장 처음부터 터지는 case 방지용 
#id=test.loc[i,'id']
for songplylst_parameter in songplylst_parameters:
  for titleplylst_parameter in titleplylst_parameters:
    for albumsong_parameter in albumsong_parameters:
      for artistsong_parameter in artistsong_parameters:
        print('now is for ',songplylst_parameter, titleplylst_parameter, albumsong_parameter, artistsong_parameter)
        answer_dicts=[]
        for i in tqdm(range(test.shape[0]),leave=True, position=0):
          if (i%100==99):
            parttag=[]
            partsong=[]
            with open('/content/drive/My Drive/Kakao Arena/test/test'+str(i-99)+'~'+str(i)+'taglist.txt', 'rb') as f:#tag
              subparttag=pickle.load(f)
            with open('/content/drive/My Drive/Kakao Arena/test/test'+str(i-99)+'~'+str(i)+'songlist.txt', 'rb') as f2:#song
              subpartsong=pickle.load(f2)
            for dicts in subparttag: #dictionary 보통 100개 불러옴
              new_dicts={}
              for key,val in dicts.items():
                if 'tagplylst' in key:
                  new_key=key.replace('tagplylst','')
                  if new_key not in new_dicts:
                    new_dicts[new_key]=tagplylst_parameter*val
                  else:
                    new_dicts[new_key]+=(tagplylst_parameter*val)
                elif 'titleplylst' in key:
                  new_key=key.replace('titleplylst','')
                  if new_key not in new_dicts:
                    new_dicts[new_key]=titleplylst_parameter*val
                  else:
                    new_dicts[new_key]+=(titleplylst_parameter*val)
                elif 'songplylst' in key:
                  new_key=key.replace('songplylst','')
                  if new_key not in new_dicts:
                    new_dicts[new_key]=songplylst_parameter*val
                  else:
                    new_dicts[new_key]+=(songplylst_parameter*val)
              parttag.append(new_dicts)
            for dicts in subpartsong: #dictionary 보통 100개 불러옴
              song_new_dicts={}
              for key,val in dicts.items():
                if 'tagplylst' in key:
                  new_key=key.replace('tagplylst','')
                  if new_key not in song_new_dicts:
                    song_new_dicts[new_key]=tagplylst_parameter*val
                  else:
                    song_new_dicts[new_key]+=(tagplylst_parameter*val)
                elif 'titleplylst' in key:
                  new_key=key.replace('titleplylst','')
                  if new_key not in song_new_dicts:
                    song_new_dicts[new_key]=titleplylst_parameter*val
                  else:
                    song_new_dicts[new_key]+=(titleplylst_parameter*val)
                elif 'songplylst' in key:
                  new_key=key.replace('songplylst','')
                  if new_key not in song_new_dicts:
                    song_new_dicts[new_key]=songplylst_parameter*val
                  else:
                    song_new_dicts[new_key]+=(songplylst_parameter*val)
                elif 'artistsong' in key:
                  new_key=key.replace('artistsong','')
                  if new_key not in song_new_dicts:
                    song_new_dicts[new_key]=artistsong_parameter*val
                  else:
                    song_new_dicts[new_key]+=(artistsong_parameter*val)
                elif 'albumsong' in key:
                  new_key=key.replace('albumsong','')
                  if new_key not in song_new_dicts:
                    song_new_dicts[new_key]=albumsong_parameter*val
                  else:
                    song_new_dicts[new_key]+=(albumsong_parameter*val)
              partsong.append(song_new_dicts)
              #이제 이게 subparttag는 키가 196264tagplaylst 이런식으로 되어있고, partsong은 196264가 들어가야한다.
            for j in range(len(parttag)):#보통 100, song이나 tag나 상관 없다.
              
              ith_answer={}
              pred_songs=[]
              pred_tags=[]
              songnum=0
              tagnum=0
              for songs in test.loc[i-len(parttag)+j,"songs"]:
                pred_songs.append(songs)
                songnum=songnum+1
              for tags in test.loc[i-len(parttag)+j,"tags"]:
                pred_tags.append(tags)
                tagnum=tagnum+1
              ressongs=dict(sorted(partsong[j].items(),key=(lambda x:x[1]),reverse=True))
              restags=dict(sorted(parttag[j].items(),key=(lambda x:x[1]),reverse=True))
              while True:
                if(songnum==100):
                  break
                #업데이트 날짜는 여기서 고려 가능
                for key in ressongs:
                  key=int(key)
                  if (key not in pred_songs):
                    pred_songs.append(key)
                    songnum+=1
                    if songnum==100:
                      break
                
                if (songnum!=100):
                  pred_songs=before_songs
                  songnum=100
                
                break
              while True:
                if(tagnum==10):
                  break
                for key in restags:
                  if key not in pred_tags:
                    pred_tags.append(key)
                    tagnum=tagnum+1
                    if (tagnum==10):
                      break
                
                if(tagnum!=10):
                  pred_tags=before_tags
                  tagnum=10
                
                break
              id=test.loc[i-len(parttag)+j,'id']
              ith_answer['id']=int(id)
              ith_answer['songs']=pred_songs
              ith_answer['tags']=pred_tags
              answer_dicts.append(ith_answer)
        with open('/content/drive/My Drive/Kakao Arena/test/test'+str(i-99)+'~'+str(i)+'taglist.txt', 'rb') as f:
          subparttag=pickle.load(f)
        with open('/content/drive/My Drive/Kakao Arena/test/test'+str(i-99)+'~'+str(i)+'songlist.txt', 'rb') as f2:
          subpartsong=pickle.load(f2)
        parttag=[]
        partsong=[]
        i=i+1
        for dicts in subparttag: #dictionary 보통 100개 불러옴
          new_dicts={}
          for key,val in dicts.items():
            if 'tagplylst' in key:
              new_key=key.replace('tagplylst','')
              if new_key not in new_dicts:
                new_dicts[new_key]=tagplylst_parameter*val
              else:
                new_dicts[new_key]+=(tagplylst_parameter*val)
            elif 'titleplylst' in key:
              new_key=key.replace('titleplylst','')
              if new_key not in new_dicts:
                new_dicts[new_key]=titleplylst_parameter*val
              else:
                new_dicts[new_key]+=(titleplylst_parameter*val)
            elif 'songplylst' in key:
              new_key=key.replace('songplylst','')
              if new_key not in new_dicts:
                new_dicts[new_key]=songplylst_parameter*val
              else:
                new_dicts[new_key]+=(songplylst_parameter*val)
          parttag.append(new_dicts)
        for dicts in subpartsong: #dictionary 보통 100개 불러옴
          song_new_dicts={}
          for key,val in dicts.items():
            if 'tagplylst' in key:
              new_key=key.replace('tagplylst','')
              if new_key not in song_new_dicts:
                song_new_dicts[new_key]=tagplylst_parameter*val
              else:
                song_new_dicts[new_key]+=(tagplylst_parameter*val)
            elif 'titleplylst' in key:
              new_key=key.replace('titleplylst','')
              if new_key not in song_new_dicts:
                song_new_dicts[new_key]=titleplylst_parameter*val
              else:
                song_new_dicts[new_key]+=(titleplylst_parameter*val)
            elif 'songplylst' in key:
              new_key=key.replace('songplylst','')
              if new_key not in song_new_dicts:
                song_new_dicts[new_key]=songplylst_parameter*val
              else:
                song_new_dicts[new_key]+=(songplylst_parameter*val)
            elif 'artistsong' in key:
              new_key=key.replace('artistsong','')
              if new_key not in song_new_dicts:
                song_new_dicts[new_key]=artistsong_parameter*val
              else:
                song_new_dicts[new_key]+=(artistsong_parameter*val)
            elif 'albumsong' in key:
              new_key=key.replace('albumsong','')
              if new_key not in song_new_dicts:
                song_new_dicts[new_key]=albumsong_parameter*val
              else:
                song_new_dicts[new_key]+=(albumsong_parameter*val)
          partsong.append(song_new_dicts)
        for j in range(len(parttag)):#보통 100, song이나 tag나 상관 없다.
          ith_answer={}
          pred_songs=[]
          pred_tags=[]
          songnum=0
          tagnum=0
          for songs in test.loc[i-len(parttag)+j,"songs"]:
            songs=int(songs)
            pred_songs.append(songs)
            songnum=songnum+1
          for tags in test.loc[i-len(parttag)+j,"tags"]:
            pred_tags.append(tags)
            tagnum=tagnum+1
          ressongs=dict(sorted(partsong[j].items(),key=(lambda x:x[1]),reverse=True))
          restags=dict(sorted(parttag[j].items(),key=(lambda x:x[1]),reverse=True))
          while True:
            if(songnum==100):
                  break
            #업데이트 날짜는 여기서 고려 가능
            for key in ressongs:
              key=int(key)
              if key not in pred_songs:
                pred_songs.append(key)
                songnum=songnum+1
                if (songnum==100):
                  break
                  
            if(songnum!=100):
              pred_songs=before_songs
              songnum=100
            
            break
          while True:
            if(tagnum==10):
                  break
            for key in restags:
              if key not in pred_tags:
                pred_tags.append(key)
                tagnum=tagnum+1
                if (tagnum==10):
                  break
            if(tagnum!=10):
              pred_tags=before_tags
              tagnum=10
            
            break
          id=test.loc[i-len(parttag)+j,'id']
          ith_answer['id']=int(id)
          ith_answer['songs']=pred_songs
          ith_answer['tags']=pred_tags
          answer_dicts.append(ith_answer)
        with open('/content/drive/My Drive/Kakao Arena/test/'+str(tagplylst_parameter)+str(songplylst_parameter)+str(titleplylst_parameter)
+str(albumsong_parameter)+str(artistsong_parameter)+'results.json', 'w',encoding='utf-8') as fp:
            json.dump(answer_dicts, fp,ensure_ascii=False)

  0%|          | 0/10740 [00:00<?, ?it/s]

now is for  30 3 3 3


  0%|          | 0/10740 [00:00<?, ?it/s]

now is for  30 3 3 150


  0%|          | 0/10740 [00:00<?, ?it/s]

...

  0%|          | 0/10740 [00:00<?, ?it/s]

now is for  30 50 150 3


  0%|          | 0/10740 [00:00<?, ?it/s]

now is for  30 50 150 150


100%|██████████| 10740/10740 [29:33<00:00,  6.05it/s]


In [ ]:
with open('/content/drive/My Drive/Kakao Arena/'+str(tagplylst_parameter)+str(songplylst_parameter)+str(titleplylst_parameter)
+str(albumsong_parameter)+str(artistsong_parameter)+'results.json', 'w') as fp:
    json.dump(answer_dicts, fp)

In [ ]:
#tot_songs[i]가 dict가 담겨있고(곡id : 숫자 형태) 숫자가 제일 높은 곡 id n개를 일단 잡으면 된다.
for i in range(test.shape[0]):
  answer_dict={}
  pred_songs=[]
  pred_tags=[]
  id=test.loc[i,'id']
  pred_songs.extend(test.loc[i,'songs'])
  pred_tags.extend(test.loc[i,'tags'])
  songs in test.loc[i,'songs']:#플레이 리스트마다 50번 도는거임. 50번 아닐수도 있더군요..
  

